In [ ]:
import pandas as pd
import json
import requests 
import geopandas as gpd
from shapely.geometry import Point, Polygon, shape
import numpy as np


In [88]:
district_boundary_api = "https://opendata.vancouver.ca/api/explore/v2.1/catalog/datasets/local-area-boundary/records?limit=20"
header =  {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"}

resp = requests.get(district_boundary_api, headers=header)
boundary_txt =  resp.text
boundary_data = json.loads(boundary_txt)

In [94]:
df = pd.read_csv('../data/processed_data/vancouver_real_estate.csv', index_col=0)
df.reset_index(inplace=True)
df.set_index('index', inplace=True)

In [95]:
def coords_to_polygon(coords):
    return Polygon(coords)

In [112]:
district_polys = []
for i in range(boundary_data['total_count']-2):
    
    district_info = boundary_data['results'][i]
    district_name = district_info['name']
    district_coords = district_info['geom']['geometry']['coordinates'][0]
    district_polygon = coords_to_polygon(district_coords)
    district_polys.append((district_name,district_polygon))

In [114]:
district = []
for i in range(10):
    if np.isnan(df.iloc[i, 7]):
        district.append(np.nan)
    else:
        poi = Point(df.iloc[i, 7], df.iloc[i,6])
        for name, poly in district_polys:
            if poly.contains(poi):
                district.append(name)
            else:
                district.append("outside of vancouver")